In [2]:

import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import nltk
from numba import jit
from gensim.models import FastText
from joblib import Parallel, delayed
from tqdm import tqdm
import string

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

EMBEDDING_FILE = '../data/crawl-300d-2M.vec'

train = pd.read_csv('../data/train.csv.zip')
train["comment_text"] = train.comment_text.apply(lambda x: x[:800])
test = pd.read_csv('../data/test.csv.zip')
test["comment_text"] = test.comment_text.apply(lambda x: x[:800])
submission = pd.read_csv('../data/sample_submission.csv.zip')

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values
pd.options.display.max_rows=500
pd.options.display.max_colwidth=2000

/media/stas/fastdata/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [35]:
train.drop(104283,inplace=True)

In [3]:
a = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [4]:
explore = pd.read_csv("../submits/gru_with_attention_cv10/training_data")

In [ ]:
train.iloc[np.argsort(np.abs(explore.identity_hate - train.identity_hate))]
#np.argsort(explore.identity_hate - train.identity_hate)

In [6]:
train.iloc[np.argsort(explore.toxic - train.toxic)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
63164,a90714b9e4137961,"You are asshole master of deletions \n 22:18, 17 March 2013 (diff | hist) . . (+685)‎ . . User talk:PatGallacher ‎ (Notifying author of deletion nomination for Patricia Walsh) (current)\n 22:18, 17 March 2013 (diff | hist) . . (+299)‎ . . Patricia Walsh ‎ (Nominated page for deletion using Page Curation (subst:blp-prod))\n 22:11, 17 March 2013 (diff | hist) . . (+184)‎ . . Wikipedia:Administrator intervention against vandalism ‎ (Reporting Novaranist. (TW))\n 22:10, 17 March 2013 (diff | hist) . . (-33)‎ . . m Hans Kohala ‎ (Reverted 1 edit by Novaranist (talk) to last revision by Severo. (TW)) (current)\n 22:08, 17 March 2013 (diff | hist) . . (-82)‎ . . Most Extreme Airports ‎ (current)\n 22:04, 17 March 2013 (diff | hist) . . (+861)‎ . . N User talk:Matt8600 ‎ (Notifying author of deletion no",1,0,1,0,0,0
154978,b7339687a601bbe7,"He is allowed to have his list. There is no direct attack. And by the way, slander is spoken, libel would be written. Perhaps YOU should study the law more. Don't try to fight with me, because I will win. 6 July 2005 19:51 (UTC)",1,0,0,0,0,0
61030,a36aa631f4e33bc7,"""\n\n HE BELIEVES IN OPENNESS BUT CLOSES HIS PAGE - BULL$HIT WE ALL SEE THRU U JIMBO \n\nWikipedia Founder Hit With Relationship Trouble, Allegations of Excessive Spending\nBy Megan McCarthy March 03, 2008 | 3:34:58 PMCategories: People \nWikipedia founder Jimmy Wales didn't have such a good weekend. First the blogosphere and then Wikipedia itself lit up with news of his messy breakup with controversial Canadian TV pundit Rachel Marsden. Now, accusations are flying that Wales has been living the high life on the expense account of the nonprofit foundation he created.\nOn Friday, reports surfaced that the married-but-divorcing Internet icon carried on a clandestine affair with Marsden. Evidence of the affair included lurid IM transcripts, which appeared on Silicon Valley gossip blog Valleywag. O",1,0,1,0,0,0
8243,15eb1135c7e08836,"Check the diff, swedefag",1,0,0,0,0,0
59077,9e39eb48b71d7f14,"(UTC)\nSo, official wikipedia's doxa is that RS'homosexuality is a calomny ??? 15:02, 15 May 2014",1,0,0,0,0,0
61221,a3de8096cc58650c,"""\n\n Hello \n\nPlease remove inapproriate language from your edit here before you want someone to discuss that with you. Thanks. - Wiki me up® \n\n OK! Great! Which part exactly is """"inappropriate"""" and by whose standards exactly? Thanks. Let me know, so that I can do whatever it is that you command / demand of me. And the words that you want removed, please let me know what I can / should / am allowed to replace them with? Thanks for your help! And your concern! Much appreciated! Thanks! ( )""",1,0,0,0,0,0
72388,c1bef83163b6adf6,"I didn't say that I myself don't agree with what the reference says, or I myself know better than what the reference says, so I am going to correct it or remove it based on my own original research. Do not distort my words. I said Myanmar has nothing to do with the topic. You have problems with understanding.",1,0,0,0,0,0
115006,66fcbd0529153014,"I AM REALLY UPSET with his deletion nominations!!!! \n\n Sorry, but you are an asshole \n\nThis is very insane to delete an article like this.\n\n You are asshole master of deletions \n 22:18, 17 March 2013 (diff | hist) . . (+685)‎ . . User talk:PatGallacher ‎ (Notifying author of deletion nomination for Patricia Walsh) (current)\n 22:18, 17 March 2013 (diff | hist) . . (+299)‎ . . Patricia Walsh ‎ (Nominated page for deletion using Page Curation (subst:blp-prod))\n 22:11, 17 March 2013 (diff | hist) . . (+184)‎ . . Wikipedia:Administrator intervention against vandalism ‎ (Reporting Novaranist. (TW))\n 22:10, 17 March 2013 (diff | hist) . . (-33)‎ . . m Hans Kohala ‎ (Reverted 1 edit by Novaranist (talk) to last revision by Severo. (TW)) (current)\n 22:08, 17 March 2013 (diff | hist) . . (-82)‎ .",1,0,0,0,0,0
67312,b41c73b85a1776af,")\n\nSorry abou

In [11]:
roc_auc_score(train.toxic,explore.toxic)

0.9817702780532964

In [9]:
explore.toxic[63164]

0.00032563882996328175

In [2]:
def lawer(sen):
    count = 0
    for i in sen:
        if i.isupper():
            count += 1
    if len(sen) == 0: 
        return sen
    if count/len(sen) > 0.2: 
        sen.append("gronker")
    return sen

In [3]:
def stamer(sen):
    return list(map(st.stem,sen))
st = nltk.stem.PorterStemmer()

In [4]:
def token_to_sen(sen):
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in sen]).strip()

In [5]:
def token(sen):
    return text.text_to_word_sequence(sen ,lower = False)

In [28]:
%%time
tr = Parallel(n_jobs=16)(delayed(token)(x) for x in tqdm(X_train))
tes = Parallel(n_jobs=16)(delayed(token)(x) for x in tqdm(X_test))
gro_tr = Parallel(n_jobs=16)(delayed(lawer)(x) for x in tqdm(tr))
gro_tes = Parallel(n_jobs=16)(delayed(lawer)(x) for x in tqdm(tes))
#lem_tr = Parallel(n_jobs=16)(delayed(stamer)(x) for x in tqdm(gro_tr))
#lem_tes = Parallel(n_jobs=16)(delayed(stamer)(x) for x in tqdm(gro_tes))
keras_ready_tr = Parallel(n_jobs=16)(delayed(token_to_sen)(x) for x in tqdm(gro_tr))
keras_ready_tes = Parallel(n_jobs=16)(delayed(token_to_sen)(x) for x in tqdm(gro_tes))

  0%|          | 0/159571 [00:00<?, ?it/s]

NameError: name 'token' is not defined

In [8]:
%%time
model = FastText(lem_tes+lem_tr, size=300, window=5, workers=16, iter=30)
model.save("../models/FastText/Fasttest2.gen")

CPU times: user 2h 8min 20s, sys: 14.6 s, total: 2h 8min 35s
Wall time: 42min 22s


In [2]:
model = FastText.load("../models/FastText/Fasttest2.gen")

In [9]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [10]:
max_features = 60000
maxlen = 800
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features, lower=False)
tokenizer.fit_on_texts(keras_ready_tr+keras_ready_tes)
X_train = tokenizer.texts_to_sequences(keras_ready_tr)
X_test = tokenizer.texts_to_sequences(keras_ready_tes)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [11]:
#def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
#embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = w2v.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [12]:
embedding_matrix.shape

(60000, 300)

In [13]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [14]:
list(range(1,10))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
batch_size = 32

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
for epoch in range(1,10):
    hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=1, validation_data=(X_val, y_val),
                     callbacks=[RocAuc], verbose=1)
    model.save(f'../models/Polled_gru_one_layer_0.2_selfwrite_Fasttext2/{epoch}epoch.h5')

    y_pred = model.predict(x_test, batch_size=1024)
    submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
    submission.to_csv(f'../submits/Polled_gru_one_layer_0.2_selfwrite_Fasttext2/{epoch}epoch.csv', index=False)

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 367s 2ms/step - loss: 0.0532 - acc: 0.9809 - val_loss: 0.0464 - val_acc: 0.9820

 ROC-AUC - epoch: 1 - score: 0.984992 

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 373s 2ms/step - loss: 0.0418 - acc: 0.9840 - val_loss: 0.0438 - val_acc: 0.9829

 ROC-AUC - epoch: 1 - score: 0.987013 

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 378s 2ms/step - loss: 0.0387 - acc: 0.9849 - val_loss: 0.0437 - val_acc: 0.9833

 ROC-AUC - epoch: 1 - score: 0.988192 

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 367s 2ms/step - loss: 0.0366 - acc: 0.9856 - val_loss: 0.0446 - val_acc: 0.9832

 ROC-AUC - epoch: 1 - score: 0.988554 

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [=====

KeyboardInterrupt: 